# Assignment

## Brief

Write the Python codes for the following questions.

## Instructions

Paste the answer as Python in the answer code section below each question.

### Question 1

Question: Implement a simple Thrift server and client that defines a `Student` struct with fields `name` (string), `age` (integer), and `courses` (list of strings). Include a service `School` with a method `enrollCourse` that takes a `Student` record and a course name, adds the course to the student's course list, and returns the updated `Student` record.

Answer:

In [ ]:
# Thrift schema (student.thrift)

# Thrift server (student_server.py)

# Thrift client (student_client.py)


In [ ]:
%%writefile ../schema/student.thrift

struct Student {
  1: required string name,
  2: optional i64 age,
  3: optional list<string> courses
}

service School {
  Student enrollCourse(1: required Student student 2: required list<string> courses)
}

In [ ]:
%%writefile ../student_thrift_server.py
import thriftpy2
student_thrift = thriftpy2.load("./schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_server

class School(object):
  def enrollCourse(self, student, courses):
    for course in courses:
      student.courses.append(course)
    return student

server = make_server(student_thrift.School, School(), client_timeout=None)
server.serve()

Run `python student_thrift_server.py` in terminal to start the server.

In [11]:
import thriftpy2
student_thrift = thriftpy2.load("../schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_client

school = make_client(student_thrift.School, timeout=None)

# Test that the method exists
print("Available methods:", [method for method in dir(school) if not method.startswith('_')])

# Create a student record
adam = student_thrift.Student(
    name="Adam", age=16, courses=["Mathematics", "History"]
)

Available methods: ['enrollCourse']


In [12]:
adam = school.enrollCourse(adam, ["coding"])
adam 

Student(name='Adam', age=16, courses=['Mathematics', 'History', 'coding'])

### Question 2

Question: Implement a simple Protocol Buffers server and client that defines a `Book` message with fields `title` (string), `author` (string), and `page_count` (integer). Include a service `Library` with a method `checkoutBook` that takes a `Book` message and returns the same `Book` message.

Answer:

In [ ]:
# Protobuf schema (book.proto)

# Protobuf server (book_server.py)

# Protobuf client (book_client.py)


In [17]:
%%writefile ../schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  optional string author = 2;
  optional int32 page_count = 3;
}

message checkoutBookRequest {
  Book book = 1;
}

service Library {
  rpc checkoutBook(checkoutBookRequest) returns (Book) {}
}

Overwriting ../schema/book.proto


run `python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/book.proto`

This will generate the following files:

`book_pb2.py`

`book_pb2_grpc.py`

In [18]:
%%writefile ../book_protobuf_server.py
from concurrent import futures
import grpc
import book_pb2_grpc


class Library(book_pb2_grpc.LibraryServicer):

  def checkoutBook(self, request, context):
    return request.book

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
book_pb2_grpc.add_LibraryServicer_to_server(
    Library(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../book_protobuf_server.py


Run `python book_protobuf_server.py` in a new terminal to start the server

In [19]:
import sys
sys.path.append('..')
import grpc
import book_pb2
import book_pb2_grpc

In [20]:
with grpc.insecure_channel('localhost:50051') as channel:
    book = book_pb2.Book(title="Frankenstein", author="Mary Shelley", page_count=352)
    stub = book_pb2_grpc.LibraryStub(channel)
    book = stub.checkoutBook(book_pb2.checkoutBookRequest(book=book))
    print(book)

title: "Frankenstein"
author: "Mary Shelley"
page_count: 352



## Submission

- Submit the URL of the GitHub Repository that contains your work to NTU black board.
- Should you reference the work of your classmate(s) or online resources, give them credit by adding either the name of your classmate or URL.